## CNN

In [2]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical

def load_images_from_directory(directory, label):
    images = []
    labels = []
    for filename in os.listdir(directory):
        if filename.endswith((".jpg", ".png", ".JPG", ".PNG")):
            img = cv2.imread(os.path.join(directory, filename))
            if img is not None:
                img = cv2.resize(img, (256, 256))  # Resize image
                images.append(img)
                labels.append(label)
    return np.array(images), np.array(labels)

# Base directory where all datasets are located
base_dir = 'Dataset'  # Assuming 'Dataset' contains all the subdirectories

# Load healthy and unhealthy images for each crop from the 'Dataset' directory
rice_healthy_images, rice_healthy_labels = load_images_from_directory(os.path.join(base_dir, 'Rice_healthy'), 0)
rice_unhealthy_images, rice_unhealthy_labels = load_images_from_directory(os.path.join(base_dir, 'Rice_unhealthy'), 1)

potato_healthy_images, potato_healthy_labels = load_images_from_directory(os.path.join(base_dir, 'Potato_healthy'), 2)
potato_unhealthy_images, potato_unhealthy_labels = load_images_from_directory(os.path.join(base_dir, 'Potato_unhealthy'), 3)

wheat_healthy_images, wheat_healthy_labels = load_images_from_directory(os.path.join(base_dir, 'Wheat_healthy'), 4)
wheat_unhealthy_images, wheat_unhealthy_labels = load_images_from_directory(os.path.join(base_dir, 'Wheat_unhealthy'), 5)

tomato_healthy_images, tomato_healthy_labels = load_images_from_directory(os.path.join(base_dir, 'Tomato_healthy'), 6)
tomato_unhealthy_images, tomato_unhealthy_labels = load_images_from_directory(os.path.join(base_dir, 'Tomato_unhealthy'), 7)

# Ensure all image arrays are non-empty before concatenation
image_arrays = [rice_healthy_images, rice_unhealthy_images, potato_healthy_images, potato_unhealthy_images,
                wheat_healthy_images, wheat_unhealthy_images, tomato_healthy_images, tomato_unhealthy_images]

label_arrays = [rice_healthy_labels, rice_unhealthy_labels, potato_healthy_labels, potato_unhealthy_labels,
                wheat_healthy_labels, wheat_unhealthy_labels, tomato_healthy_labels, tomato_unhealthy_labels]

# Remove any empty arrays (in case some directories don't contain valid images)
image_arrays = [arr for arr in image_arrays if len(arr) > 0]
label_arrays = [arr for arr in label_arrays if len(arr) > 0]

# Concatenate the loaded images and labels
X = np.concatenate(image_arrays)
y = np.concatenate(label_arrays)

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the images
X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0

# One-hot encode the labels
y_train = to_categorical(y_train, num_classes=8)
y_val = to_categorical(y_val, num_classes=8)

# Build CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(8, activation='softmax')  # Output layer for 8 classes
])

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(
    X_train, 
    y_train, 
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=32
)





/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
190/190 ━━━━━━━━━━━━━━━━━━━━ 158s 821ms/step - accuracy: 0.5236 - loss: 1.4456 - val_accuracy: 0.8555 - val_loss: 0.4237
Epoch 2/10
190/190 ━━━━━━━━━━━━━━━━━━━━ 216s 1s/step - accuracy: 0.8573 - loss: 0.3789 - val_accuracy: 0.8252 - val_loss: 0.4193
Epoch 3/10
190/190 ━━━━━━━━━━━━━━━━━━━━ 188s 988ms/step - accuracy: 0.9042 - loss: 0.2553 - val_accuracy: 0.8569 - val_loss: 0.3567
Epoch 4/10
190/190 ━━━━━━━━━━━━━━━━━━━━ 187s 983ms/step - accuracy: 0.9194 - loss: 0.2191 - val_accuracy: 0.8885 - val_loss: 0.3050
Epoch 5/10
190/190 ━━━━━━━━━━━━━━━━━━━━ 188s 990ms/step - accuracy: 0.9498 - loss: 0.1346 - val_accuracy: 0.8536 - val_loss: 0.4731
Epoch 6/10
190/190 ━━━━━━━━━━━━━━━━━━━━ 190s 996ms/step - accuracy: 0.9677 - loss: 0.0786 - val_accuracy: 0.8905 - val_loss: 0.3570
Epoch 7/10
190/190 ━━━━━━━━━━━━━━━━━━━━ 192s 1s/step - accuracy: 0.9726 - loss: 0.0832 - val_accuracy: 0.8945 - val_loss: 0.3593
Epoch 8/10
190/190 ━━━━━━━━━━━━━━━━━━━━ 185s 972ms/step - accuracy: 0.9821 - loss:

In [5]:
# Evaluate model
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Validation accuracy: {accuracy:.2f}')




48/48 ━━━━━━━━━━━━━━━━━━━━ 8s 170ms/step - accuracy: 0.9001 - loss: 0.4350
Validation accuracy: 0.91


## ResNet50

In [4]:
import os
import cv2
import numpy as np
import streamlit as st
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

def load_images_from_directory(directory, label):
    images = []
    labels = []
    for filename in os.listdir(directory):
        if filename.lower().endswith((".jpg", ".png", ".jpeg")):
            img = cv2.imread(os.path.join(directory, filename))
            if img is not None:
                img = cv2.resize(img, (256, 256))  # Resize image
                images.append(img)
                labels.append(label)
    return np.array(images), np.array(labels)

# Base directory where all datasets are located
base_dir = 'Dataset'  # Update this path accordingly

# Define labels for multi-class classification
label_mapping = {
    'Rice_healthy': 0,
    'Rice_unhealthy': 1,
    'Potato_healthy': 2,
    'Potato_unhealthy': 3,
    'Wheat_healthy': 4,
    'Wheat_unhealthy': 5,
    'Tomato_healthy': 6,
    'Tomato_unhealthy': 7
}

# Load images and labels
image_arrays = []
label_arrays = []
for folder, label in label_mapping.items():
    images, labels = load_images_from_directory(os.path.join(base_dir, folder), label)
    if len(images) > 0:
        image_arrays.append(images)
        label_arrays.append(labels)

# Concatenate all images and labels
X = np.concatenate(image_arrays)
y = np.concatenate(label_arrays)

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Normalize the images
X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0

# One-hot encode the labels
y_train = to_categorical(y_train, num_classes=8)
y_val = to_categorical(y_val, num_classes=8)

# Compute class weights to handle class imbalance
y_integers = np.argmax(y_train, axis=1)
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_integers), y=y_integers)
class_weights = dict(enumerate(class_weights))

# Build the model using transfer learning with ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(8, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
lr_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                 patience=3, 
                                 factor=0.5, 
                                 min_lr=1e-6)
early_stop = EarlyStopping(monitor='val_accuracy', 
                           patience=5, 
                           restore_best_weights=True)

# Train the model
history = model.fit(
    X_train, 
    y_train, 
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=32,
    class_weight=class_weights,
    callbacks=[lr_reduction, early_stop]
)

# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Validation accuracy: {accuracy:.2f}')

# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Accuracy Over Epochs')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss Over Epochs')

plt.show()

# Generate confusion matrix and classification report
y_pred = model.predict(X_val)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_val, axis=1)

cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=True, fmt='d', 
            xticklabels=label_mapping.keys(), 
            yticklabels=label_mapping.keys(), cmap='Blues')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()

print(classification_report(y_true, y_pred_classes, target_names=label_mapping.keys()))

# Save the trained model
model.save('crop_health_model.h5')


Epoch 1/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 267s 1s/step - accuracy: 0.4691 - loss: 1.5349 - val_accuracy: 0.4352 - val_loss: 1.9373 - learning_rate: 0.0010
Epoch 2/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 312s 2s/step - accuracy: 0.6663 - loss: 1.0103 - val_accuracy: 0.5236 - val_loss: 1.1034 - learning_rate: 0.0010
Epoch 3/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 1961s 11s/step - accuracy: 0.7038 - loss: 0.9198 - val_accuracy: 0.3941 - val_loss: 1.8846 - learning_rate: 0.0010
Epoch 4/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 12880s 71s/step - accuracy: 0.7315 - loss: 0.8645 - val_accuracy: 0.3557 - val_loss: 2.2393 - learning_rate: 0.0010
Epoch 5/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 6998s 38s/step - accuracy: 0.7458 - loss: 0.7699 - val_accuracy: 0.3448 - val_loss: 2.0001 - learning_rate: 0.0010
Epoch 6/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 267s 1s/step - accuracy: 0.7891 - loss: 0.6663 - val_accuracy: 0.6621 - val_loss: 0.8220 - learning_rate: 5.0000e-04
Epoch 7/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 309s 2s/step - accuracy: 0.7835

/var/folders/ms/01004vks0k52h8c4djzfdyk80000gn/T/ipykernel_36948/1178361882.py:131: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


46/46 ━━━━━━━━━━━━━━━━━━━━ 2240s 50s/step


/var/folders/ms/01004vks0k52h8c4djzfdyk80000gn/T/ipykernel_36948/1178361882.py:146: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


                  precision    recall  f1-score   support

    Rice_healthy       0.58      0.80      0.67       100
  Rice_unhealthy       1.00      0.01      0.02       101
  Potato_healthy       0.90      0.57      0.70       174
Potato_unhealthy       1.00      0.16      0.27       161
   Wheat_healthy       0.62      0.77      0.69       121
 Wheat_unhealthy       0.70      0.77      0.74        92
  Tomato_healthy       0.88      0.95      0.91       318
Tomato_unhealthy       0.62      0.93      0.74       392

        accuracy                           0.71      1459
       macro avg       0.79      0.62      0.59      1459
    weighted avg       0.78      0.71      0.66      1459



## MobileNetV2

In [6]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

# Load a pre-trained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze the base model (no training on the pre-trained layers)
base_model.trainable = False

# Add new layers on top of the pre-trained model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(8, activation='softmax')  # Assuming 8 classes (4 crops, each with healthy/unhealthy)
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10)


/var/folders/ms/01004vks0k52h8c4djzfdyk80000gn/T/ipykernel_36948/1446980196.py:5: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(256, 256, 3))


Epoch 1/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 88s 450ms/step - accuracy: 0.7258 - loss: 0.7593 - val_accuracy: 0.9157 - val_loss: 0.2139
Epoch 2/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 68s 371ms/step - accuracy: 0.9018 - loss: 0.2231 - val_accuracy: 0.9260 - val_loss: 0.1853
Epoch 3/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 68s 369ms/step - accuracy: 0.9237 - loss: 0.1854 - val_accuracy: 0.9294 - val_loss: 0.1734
Epoch 4/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 67s 367ms/step - accuracy: 0.9339 - loss: 0.1593 - val_accuracy: 0.9315 - val_loss: 0.1608
Epoch 5/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 70s 382ms/step - accuracy: 0.9477 - loss: 0.1284 - val_accuracy: 0.9328 - val_loss: 0.1758
Epoch 6/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 70s 384ms/step - accuracy: 0.9574 - loss: 0.1064 - val_accuracy: 0.9390 - val_loss: 0.1566
Epoch 7/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 74s 404ms/step - accuracy: 0.9628 - loss: 0.1016 - val_accuracy: 0.9315 - val_loss: 0.1657
Epoch 8/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 77s 422ms/step - accuracy: 0.9608 - loss: 0

In [8]:
# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Validation accuracy: {accuracy:.2f}')


46/46 ━━━━━━━━━━━━━━━━━━━━ 13s 273ms/step - accuracy: 0.9365 - loss: 0.1753
Validation accuracy: 0.94


In [10]:
model.save("f2.h5")